<a href="https://colab.research.google.com/github/omkarpat/EmpatheticDialoguesEmotionDetection/blob/master/Baselines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Colab settings/mount
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/My\ Drive/CSE\ 245\ Project

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/CSE 245 Project


In [3]:
!ls Data

'Data Exploration.ipynb'
 data_fixed_train.json
 data_sample_100.json
 data_sample_10.json
 data_sample_10_processed.json
 data_sample_fixed_processed_final.json
 data_sample_fixed_processed_model1_final.csv
 data_sample_fixed_processed_model2_final.csv
 data_sample_fixed_processed_model3_final.csv
 data_sample_fixed_processed_model4_final.csv
 fixed
 fixed_test.json
 fixed_train_516.csv
 fixed_valid.json
'informative words.ipynb'
 Raw
 test_fixed_processed_model1_final.csv
 test_fixed_processed_model2_final.csv
 test_fixed_processed_model3_final.csv
 test_fixed_processed_model4_final.csv
 valid.csv
 valid_fixed_processed_model1_final.csv
 valid_fixed_processed_model2_final.csv
 valid_fixed_processed_model3_final.csv
 valid_fixed_processed_model4_final.csv


In [4]:
import nltk
nltk.download('vader_lexicon')
!pip install textblob
!pip install flair

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
     |████████████████████████████████| 143kB 2.8MB/s 
     |████████████████████████████████| 256kB 7.5MB/s 
     |████████████████████████████████| 665kB 12.2MB/s 
     |████████████████████████████████| 798kB 17.5MB/s 
     |████████████████████████████████| 983kB 20.8MB/s 
     |████████████████████████████████| 890kB 23.1MB/s 
     |████████████████████████████████| 3.8MB 31.2MB/s 
     |████████████████████████████████| 1.1MB 54.7MB/s 
  Created wheel for segtok: filename=segtok-1.5.10-cp36-none-any.whl size=25020 sha256=0acc4533166f323474e47e20c888b3e70db9db8dde5afda4d3fd2eace40105a5
  Stored in directory: /root/.cache/pip/wheels/b4/39/f6/9ca1c5cabde964d728023b5751c3a206a5c8cc40252321fb6b
  Created wheel for mpld3: filename=mpld3-0.3-cp36-none-any.whl size=116679 sha256=891a09429821d4f6006346b623eb78992625a859dda5b6c8e5652f45619dbb12
  Stored in directory: /root/.cache/pip/wheels/c0/47/fb/8a64f89aecfe00598304793

In [5]:
import json
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob
import flair
import torch
torch.__version__ = '1.5.0'

/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [6]:
vader = SentimentIntensityAnalyzer()
flair_sentiment = flair.models.TextClassifier.load('en-sentiment')

2020-05-23 03:28:35,910 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/models-v0.4/classy-imdb-en-rnn-cuda%3A0/imdb-v0.4.pt not found in cache, downloading to /tmp/tmpffywq47n


100%|██████████| 1501979561/1501979561 [01:02<00:00, 24036028.44B/s]

2020-05-23 03:29:38,954 copying /tmp/tmpffywq47n to cache at /root/.flair/models/imdb-v0.4.pt


2020-05-23 03:29:45,213 removing temp file /tmp/tmpffywq47n
2020-05-23 03:29:45,347 loading file /root/.flair/models/imdb-v0.4.pt


In [9]:
train_data = json.load(open("Data/fixed_valid.json"))
print(len(train_data.keys()))

2763


In [10]:
results_data = {}
train_data_model1, train_data_model2, train_data_model3, train_data_model4 = [], [], [], []
for i,key in enumerate(train_data.keys()):
    speaker = train_data[key]['dialog'][0]['speaker']
                             
    train_data_model1_string = str(train_data[key]["prompt"]).replace("|","")
    train_data_model2_string = str(train_data[key]["prompt"]).replace("|","") + " " + str(train_data[key]['dialog'][0]['utterance']).replace("|","")
    train_data_model3_string = str(train_data[key]["prompt"]).replace("|","")
    train_data_model4_string = str(train_data[key]["prompt"]).replace("|","")
    
    for item in train_data[key]['dialog']:
        train_data_model4_string += " " + str(item['utterance']).replace("|","")
        if item['speaker'] == speaker:
            train_data_model3_string += " " + str(item['utterance']).replace("|","")
    
    s1 = flair.data.Sentence(train_data_model1_string)
    s2 = flair.data.Sentence(train_data_model2_string)
    s3 = flair.data.Sentence(train_data_model3_string)
    s4 = flair.data.Sentence(train_data_model4_string)
    
    flair_sentiment.predict(s1)
    flair_sentiment.predict(s2)
    flair_sentiment.predict(s3)
    flair_sentiment.predict(s4)
    
    
    results_data[key] = {"emotion": train_data[key]["emotion"], "results": {
      "model1" : {"input": train_data_model1_string, "vader": vader.polarity_scores(train_data_model1_string), 
                  "textblob": TextBlob(train_data_model1_string).sentiment.polarity, "flair":s1.labels[0].to_dict()}, 
      "model2" : {"input": train_data_model2_string, "vader": vader.polarity_scores(train_data_model2_string), 
                  "textblob": TextBlob(train_data_model2_string).sentiment.polarity, "flair":s2.labels[0].to_dict()}, 
      "model3" : {"input": train_data_model3_string, "vader": vader.polarity_scores(train_data_model3_string), 
                  "textblob": TextBlob(train_data_model3_string).sentiment.polarity, "flair":s3.labels[0].to_dict()}, 
      "model4" : {"input": train_data_model4_string, "vader": vader.polarity_scores(train_data_model4_string), 
                  "textblob": TextBlob(train_data_model4_string).sentiment.polarity, "flair":s4.labels[0].to_dict()}
      }}
    
    #emotion|processed_emotion|text|vader_neg|vader_neu|vader_pos|vader_compound|textblob|flair_value|flair_confidence

    line_array_model1, line_array_model2, line_array_model3, line_array_model4 = [], [], [], []

    line_array_model1.append(train_data[key]["emotion"])
    if (train_data[key]["emotion"]).lower() in ["disgusted","devastated", "terrified", "anxious", "furious", "embarrassed",
                      "angry", "sad", "afraid", "annoyed", "jealous", "ashamed", "lonely", "guilty",
                      "apprehensive", "disappointed"]:
        line_array_model1.append("negative")
    else:
        line_array_model1.append("positive")
    line_array_model1.append(train_data_model1_string)
    line_array_model1.append(str(vader.polarity_scores(train_data_model1_string)["neg"]))
    line_array_model1.append(str(vader.polarity_scores(train_data_model1_string)["neu"]))
    line_array_model1.append(str(vader.polarity_scores(train_data_model1_string)["pos"]))
    line_array_model1.append(str(vader.polarity_scores(train_data_model1_string)["compound"]))
    line_array_model1.append(str(TextBlob(train_data_model1_string).sentiment.polarity))
    line_array_model1.append(str(s1.labels[0].to_dict()["value"]))
    line_array_model1.append(str(s1.labels[0].to_dict()["confidence"]))
                        
    line_array_model2.append(train_data[key]["emotion"])
    if (train_data[key]["emotion"]).lower() in ["disgusted","devastated", "terrified", "anxious", "furious", "embarrassed",
                      "angry", "sad", "afraid", "annoyed", "jealous", "ashamed", "lonely", "guilty",
                      "apprehensive", "disappointed"]:
        line_array_model2.append("negative")
    else:
        line_array_model2.append("positive")
    line_array_model2.append(train_data_model2_string)
    line_array_model2.append(str(vader.polarity_scores(train_data_model2_string)["neg"]))
    line_array_model2.append(str(vader.polarity_scores(train_data_model2_string)["neu"]))
    line_array_model2.append(str(vader.polarity_scores(train_data_model2_string)["pos"]))
    line_array_model2.append(str(vader.polarity_scores(train_data_model2_string)["compound"]))
    line_array_model2.append(str(TextBlob(train_data_model2_string).sentiment.polarity))
    line_array_model2.append(str(s2.labels[0].to_dict()["value"]))
    line_array_model2.append(str(s2.labels[0].to_dict()["confidence"]))

    line_array_model3.append(train_data[key]["emotion"])
    if (train_data[key]["emotion"]).lower() in ["disgusted","devastated", "terrified", "anxious", "furious", "embarrassed",
                      "angry", "sad", "afraid", "annoyed", "jealous", "ashamed", "lonely", "guilty",
                      "apprehensive", "disappointed"]:
        line_array_model3.append("negative")
    else:
        line_array_model3.append("positive")
    line_array_model3.append(train_data_model3_string.replace("|",""))
    line_array_model3.append(str(vader.polarity_scores(train_data_model3_string)["neg"]))
    line_array_model3.append(str(vader.polarity_scores(train_data_model3_string)["neu"]))
    line_array_model3.append(str(vader.polarity_scores(train_data_model3_string)["pos"]))
    line_array_model3.append(str(vader.polarity_scores(train_data_model3_string)["compound"]))
    line_array_model3.append(str(TextBlob(train_data_model3_string).sentiment.polarity))
    line_array_model3.append(str(s3.labels[0].to_dict()["value"]))
    line_array_model3.append(str(s3.labels[0].to_dict()["confidence"]))
                        
    line_array_model4.append(train_data[key]["emotion"])
    if (train_data[key]["emotion"]).lower() in ["disgusted","devastated", "terrified", "anxious", "furious", "embarrassed",
                      "angry", "sad", "afraid", "annoyed", "jealous", "ashamed", "lonely", "guilty",
                      "apprehensive", "disappointed"]:
        line_array_model4.append("negative")
    else:
        line_array_model4.append("positive")
    line_array_model4.append(train_data_model4_string)
    line_array_model4.append(str(vader.polarity_scores(train_data_model4_string)["neg"]))
    line_array_model4.append(str(vader.polarity_scores(train_data_model4_string)["neu"]))
    line_array_model4.append(str(vader.polarity_scores(train_data_model4_string)["pos"]))
    line_array_model4.append(str(vader.polarity_scores(train_data_model4_string)["compound"]))
    line_array_model4.append(str(TextBlob(train_data_model4_string).sentiment.polarity))
    line_array_model4.append(str(s4.labels[0].to_dict()["value"]))
    line_array_model4.append(str(s4.labels[0].to_dict()["confidence"]))

    train_data_model1.append("|".join(line_array_model1))
    train_data_model2.append("|".join(line_array_model2))
    train_data_model3.append("|".join(line_array_model3))
    train_data_model4.append("|".join(line_array_model4))

    if i%1000 == 999:
      print(i)
      '''
      filename = "Data/data_sample_fixed_processed_" + str(i) + ".json"
      with open(filename, 'w') as outfile:
          outfile.write(json.dumps(results_data, indent=4))
          results_data = {}

      filename = "Data/data_sample_fixed_processed_" + "model1_" + str(i) + ".csv"
      with open(filename, 'w') as outfile:
          outfile.write("emotion|processed_emotion|text|vader_neg|vader_neu|vader_pos|vader_compound|textblob|flair_value|flair_confidence\n")
          outfile.write("\n".join(train_data_model1))
                            
      filename = "Data/data_sample_fixed_processed_" + "model2_" + str(i) + ".csv"
      with open(filename, 'w') as outfile:
          outfile.write("emotion|processed_emotion|text|vader_neg|vader_neu|vader_pos|vader_compound|textblob|flair_value|flair_confidence\n")
          outfile.write("\n".join(train_data_model2))
                                  
      filename = "Data/data_sample_fixed_processed_" + "model3_" + str(i) + ".csv"
      with open(filename, 'w') as outfile:
          outfile.write("emotion|processed_emotion|text|vader_neg|vader_neu|vader_pos|vader_compound|textblob|flair_value|flair_confidence\n")
          for line in train_data_model3:
              try:
                  outfile.write(line)
                  outfile.write("\n")
              except UnicodeEncodeError:
                  print(line)
                                  
      filename = "Data/data_sample_fixed_processed_" + "model4_" + str(i) + ".csv"
      with open(filename, 'w') as outfile:
          outfile.write("emotion|processed_emotion|text|vader_neg|vader_neu|vader_pos|vader_compound|textblob|flair_value|flair_confidence\n")
          for line in train_data_model4:
              try:
                  outfile.write(line)
                  outfile.write("\n")
              except UnicodeEncodeError:
                  print(line)
    '''    
filename = "Data/valid_fixed_processed_final.json"
with open(filename, 'w') as outfile:
    outfile.write(json.dumps(results_data, indent=4))
    results_data = {}
filename = "Data/valid_fixed_processed_" + "model1_" + "final.csv"
with open(filename, 'w') as outfile:
    outfile.write("emotion|processed_emotion|text|vader_neg|vader_neu|vader_pos|vader_compound|textblob|flair_value|flair_confidence\n")
    outfile.write("\n".join(train_data_model1))
                            
filename = "Data/valid_fixed_processed_" + "model2_" + "final.csv"
with open(filename, 'w') as outfile:
    outfile.write("emotion|processed_emotion|text|vader_neg|vader_neu|vader_pos|vader_compound|textblob|flair_value|flair_confidence\n")
    outfile.write("\n".join(train_data_model2))
                           
filename = "Data/valid_fixed_processed_" + "model3_" + "final.csv"
with open(filename, 'w') as outfile:
    outfile.write("emotion|processed_emotion|text|vader_neg|vader_neu|vader_pos|vader_compound|textblob|flair_value|flair_confidence\n")
    for line in train_data_model3:
        try:
            outfile.write(line)
            outfile.write("\n")
        except UnicodeEncodeError:
            print(line)
                            
filename = "Data/valid_fixed_processed_" + "model4_" + "final.csv"
with open(filename, 'w') as outfile:
    outfile.write("emotion|processed_emotion|text|vader_neg|vader_neu|vader_pos|vader_compound|textblob|flair_value|flair_confidence\n")
    for line in train_data_model4:
        try:
            outfile.write(line)
            outfile.write("\n")
        except UnicodeEncodeError:
            print(line)


999
1999


In [0]:
print(train_data_model1)
#print(train_data_model2[0])
#print(train_data_model3[0])
#print(train_data_model4[0])

['0.127', '0.722', '0.151', '0.1119', '-0.20357142857142857', 'sad|sad|We (family friends) planned a holiday trip to go for the last weekend. I was preparing for that like packing dress and getting ready to it. Unfortunately one of my cousin was unable to come due to his personal reasons. So we just cancelled the trip. I was feeling sad for this trip cancellation. ', '0.0', '0.72', '0.28', '0.5413', '0.25', 'jealous|jealous|My cousin bought a really pretty shirt that I had been wanting.', '0.228', '0.685', '0.088', '-0.6256', '-0.012500000000000011', "jealous|jealous|I was at the store and I tried on a dress_comma_ and it didn't look good. I saw someone else wearing it_comma_ looking great_comma_ and it made me feel pretty insecure and sad. ]", '0.0', '1.0', '0.0', '0.0', '0.0', 'caring|caring|I was outside walking one day. And then I found a kitten on the road.', '0.07', '0.553', '0.377', '0.7964', '0.18958333333333333', 'proud|proud|I won first place in the marathon last weekend. I t

In [0]:
#print(json.dumps(results_data['1022'], indent=4))
with open('Data/data_sample_100_processed.json', 'w') as outfile:
    outfile.write(json.dumps(results_data, indent=4))

In [0]:
emotion|processed_emotion|text|vader_neg|vader_neu|vader_pos|vader_compound|textblob|flair_value|flair_confidence|model_value|model_confidence

In [0]:
import torch
print(torch.__version__)

1.4.0
